In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
#GPU libraries
import cudf as pd
import cupy as cp
import cuml as np
from cuml import LinearRegression
from cuml.linear_model import LinearRegression
from cuml import Ridge
from cuml.linear_model import Ridge
from cuml.model_selection import train_test_split
from cuml.linear_model import Lasso
from cuml.ensemble import RandomForestRegressor

In [4]:
#Evaluation Metrics
import seaborn as sns
import sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, Normalizer, StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, clone
import sklearn_pandas
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score,train_test_split
from scipy import stats
# from sklearn.linear_model import LinearRegression
from scipy.special import boxcox1p
import csv



In [5]:
#Environment Specs
import sys
import scipy

print('Environment specification:\n')
print('python', '%s.%s.%s' % sys.version_info[:3])

for mod in np, scipy, sns, sklearn, pd:
    print(mod.__name__, mod.__version__)

In [6]:
# kaggle train data
data_df = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
data_df

In [7]:
#kaggle test data 
test_df = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
test_df


In [8]:
data_df.info()


In [9]:
test_df.info()

In [10]:
data_df.describe()

In [11]:
data_df['SalePrice'].describe()

In [12]:
temp = data_df
temp = temp.to_pandas()
sns.boxplot(temp['SalePrice'])


In [13]:
sns.distplot(temp['SalePrice'])

In [14]:
import matplotlib.pyplot as plt

col_nan = temp.isna().sum() / temp.shape[0]
plt.figure(figsize=(8, 5))
sns.set(font_scale=1.2)
col_nan[col_nan > 0.01].plot(kind = "barh")
plt.title("Features with the highest percentage of Nan values")



In [15]:
test_df.isnull().sum().sort_values(ascending=False).iloc[:35]

In [16]:
data_df.isnull().sum().sort_values(ascending=False).iloc[:20]

In [17]:
## Top 5 Fields of test data having null values
print(test_df.isnull().sum().sort_values(ascending=False).iloc[:5])
## Top 5 Fields of test data having null values
print(data_df.isnull().sum().sort_values(ascending=False).iloc[:5])

In [18]:
#Here we are getting a lot of features with a lot of missing values 
data_df.isnull().sum()

In [19]:
num_train = data_df.shape[0]
print("No. of rows of train_df= " ,num_train)

In [20]:
num_test = test_df.shape[0]
print("No. of rows test_df = ",num_test)


In [21]:
print("Columns of train_df =",data_df.columns)

In [22]:
Y = data_df['SalePrice']

In [23]:
data_df.drop('SalePrice', axis = 1 , inplace = True)




In [24]:
data_df.columns


In [25]:
numerical_features = data_df.dtypes[data_df.dtypes != 'object'].index
print(numerical_features)



In [26]:
categorial_features = data_df.dtypes[data_df.dtypes == 'object'].index
print(categorial_features)



In [27]:
print("Number of numerical_features = " , len(numerical_features))
print("Number of categorial_features = " , len(categorial_features))


## HANDLING THE NULL VALUES


In [28]:
data_df['LotFrontage']= data_df['LotFrontage'].fillna(data_df['LotFrontage'].mean())
data_df['LotFrontage'].value_counts()


In [29]:
test_df['LotFrontage']= test_df['LotFrontage'].fillna(test_df['LotFrontage'].mean())
test_df['LotFrontage'].value_counts()

In [30]:
#Handling Categorical Features for Training Data
data_df['BsmtCond']=data_df['BsmtCond'].fillna(data_df['BsmtCond'].mode()[0])
data_df['BsmtQual']=data_df['BsmtQual'].fillna(data_df['BsmtQual'].mode()[0])
data_df['FireplaceQu']=data_df['FireplaceQu'].fillna(data_df['FireplaceQu'].mode()[0])
data_df['GarageType']=data_df['GarageType'].fillna(data_df['GarageType'].mode()[0])
data_df['GarageFinish']=data_df['GarageFinish'].fillna(data_df['GarageFinish'].mode()[0])
data_df['GarageQual']=data_df['GarageQual'].fillna(data_df['GarageQual'].mode()[0])
data_df['GarageCond']=data_df['GarageCond'].fillna(data_df['GarageCond'].mode()[0])
data_df['MasVnrType']=data_df['MasVnrType'].fillna(data_df['MasVnrType'].mode()[0])
data_df['MasVnrArea']=data_df['MasVnrArea'].fillna(data_df['MasVnrArea'].mode()[0])
data_df['BsmtExposure']=data_df['BsmtExposure'].fillna(data_df['BsmtExposure'].mode()[0])
data_df['BsmtFinType2']=data_df['BsmtFinType2'].fillna(data_df['BsmtFinType2'].mode()[0])
data_df['Utilities']=data_df['Utilities'].fillna(data_df['Utilities'].mode()[0])
data_df['Exterior1st']=data_df['Exterior1st'].fillna(data_df['Exterior1st'].mode()[0])
data_df['Exterior2nd']=data_df['Exterior2nd'].fillna(data_df['Exterior2nd'].mode()[0])
data_df['BsmtFinType1']=data_df['BsmtFinType1'].fillna(data_df['BsmtFinType1'].mode()[0])
data_df['SaleType']=data_df['SaleType'].fillna(data_df['SaleType'].mode()[0])
data_df['BsmtFullBath']=data_df['BsmtFullBath'].fillna(data_df['BsmtFullBath'].mode()[0])
data_df['BsmtHalfBath']=data_df['BsmtHalfBath'].fillna(data_df['BsmtHalfBath'].mode()[0])
data_df['KitchenQual']=data_df['KitchenQual'].fillna(data_df['KitchenQual'].mode()[0])
data_df['Functional']=data_df['Functional'].fillna(data_df['Functional'].mode()[0])
data_df['GarageCars']=data_df['GarageCars'].fillna(data_df['GarageCars'].mean())
data_df['GarageArea']=data_df['GarageArea'].fillna(data_df['GarageArea'].mean())
data_df['BsmtFinSF1']=data_df['BsmtFinSF1'].fillna(data_df['BsmtFinSF1'].mean())
data_df['BsmtFinSF2']=data_df['BsmtFinSF2'].fillna(data_df['BsmtFinSF2'].mean())
data_df['BsmtUnfSF']=data_df['BsmtUnfSF'].fillna(data_df['BsmtUnfSF'].mean())
data_df['TotalBsmtSF']=data_df['TotalBsmtSF'].fillna(data_df['TotalBsmtSF'].mean())

In [31]:
#Handling Categorical Features for Test Data
test_df['BsmtCond']=test_df['BsmtCond'].fillna(test_df['BsmtCond'].mode()[0])
test_df['BsmtQual']=test_df['BsmtQual'].fillna(test_df['BsmtQual'].mode()[0])
test_df['FireplaceQu']=test_df['FireplaceQu'].fillna(test_df['FireplaceQu'].mode()[0])
test_df['GarageType']=test_df['GarageType'].fillna(test_df['GarageType'].mode()[0])
test_df['GarageFinish']=test_df['GarageFinish'].fillna(test_df['GarageFinish'].mode()[0])
test_df['GarageQual']=test_df['GarageQual'].fillna(test_df['GarageQual'].mode()[0])
test_df['GarageCond']=test_df['GarageCond'].fillna(test_df['GarageCond'].mode()[0])
test_df['MasVnrType']=test_df['MasVnrType'].fillna(test_df['MasVnrType'].mode()[0])
test_df['MasVnrArea']=test_df['MasVnrArea'].fillna(test_df['MasVnrArea'].mode()[0])
test_df['BsmtExposure']=test_df['BsmtExposure'].fillna(test_df['BsmtExposure'].mode()[0])
test_df['BsmtFinType2']=test_df['BsmtFinType2'].fillna(test_df['BsmtFinType2'].mode()[0])
test_df['MSZoning']=test_df['MSZoning'].fillna(test_df['MSZoning'].mode()[0])
test_df['Utilities']=test_df['Utilities'].fillna(test_df['Utilities'].mode()[0])
test_df['Exterior1st']=test_df['Exterior1st'].fillna(test_df['Exterior1st'].mode()[0])
test_df['Exterior2nd']=test_df['Exterior2nd'].fillna(test_df['Exterior2nd'].mode()[0])
test_df['BsmtFinType1']=test_df['BsmtFinType1'].fillna(test_df['BsmtFinType1'].mode()[0])
test_df['SaleType']=test_df['SaleType'].fillna(test_df['SaleType'].mode()[0])
test_df['BsmtFullBath']=test_df['BsmtFullBath'].fillna(test_df['BsmtFullBath'].mode()[0])
test_df['BsmtHalfBath']=test_df['BsmtHalfBath'].fillna(test_df['BsmtHalfBath'].mode()[0])
test_df['KitchenQual']=test_df['KitchenQual'].fillna(test_df['KitchenQual'].mode()[0])
test_df['Functional']=test_df['Functional'].fillna(test_df['Functional'].mode()[0])
test_df['GarageCars']=test_df['GarageCars'].fillna(test_df['GarageCars'].mean())
test_df['GarageArea']=test_df['GarageArea'].fillna(test_df['GarageArea'].mean())
test_df['BsmtFinSF1']=test_df['BsmtFinSF1'].fillna(test_df['BsmtFinSF1'].mean())
test_df['BsmtFinSF2']=test_df['BsmtFinSF2'].fillna(test_df['BsmtFinSF2'].mean())
test_df['BsmtUnfSF']=test_df['BsmtUnfSF'].fillna(test_df['BsmtUnfSF'].mean())
test_df['TotalBsmtSF']=test_df['TotalBsmtSF'].fillna(test_df['TotalBsmtSF'].mean())

In [32]:
data_df

In [33]:
test_df

In [34]:
data_df.isnull().sum()


In [35]:
test_df.isnull().sum()

In [36]:
#dropping all the values less than 50%
data_df.drop(['Alley' , 'PoolQC' , 'Fence' , 'MiscFeature' , 'GarageYrBlt' , 'Id'] , axis = 1 , inplace = True)

In [37]:
test_df.drop(['Alley' , 'PoolQC' , 'Fence' , 'MiscFeature' , 'GarageYrBlt' , 'Id'] , axis = 1 , inplace = True)

In [38]:
#AGAIN WE WILL VISUALIZE THE DATA AND PLOT, WITH CLEANER SEGMENT
train1 = data_df
train1 = train1.to_pandas()
sns.heatmap(train1.isnull() )

In [39]:
data_df.isnull().any().any()


In [40]:
test_df.isnull().any().any()


In [41]:
final_df = pd.concat([data_df , test_df] , axis = 0)

In [42]:
final_df


In [43]:
final_df.shape


In [44]:
len(final_df)

In [45]:
final_df = pd.get_dummies(final_df)
final_df

In [46]:
#splitting data into train , test 
train_df = final_df.iloc[:num_train , :]
test_df = final_df.iloc[num_test+1 :, :]

In [47]:
X= train_df

In [48]:
X.shape

In [49]:
Y.shape

## CREATING THE MODELS AND APPLICATION OF THE 5 REGRESSION ALGOS


In [50]:
X_train , X_test , y_train , y_test = train_test_split(X , Y , test_size=0.29 , random_state = 42)

In [51]:
X.isnull().any().any

In [52]:
#creating the model and  fitting it
linreg = LinearRegression(fit_intercept = True , normalize = False , algorithm = 'svd-jacobi')

In [53]:
split = linreg.fit(X,Y)

In [54]:
print("R-Squared Value for Training Set: {:.3f}".format(linreg.score(X_train.astype('float32'), y_train.astype('float32'))))

In [55]:
test_df.shape

In [56]:
y_pred_split=linreg.predict(test_df)
print(y_pred_split)
print(y_pred_split.shape)

In [57]:
algorithm = ['svd', 'eig', 'svd-qr', 'svd-jacobi','qr']
#these are the 5 algos

In [58]:
import cuml
#Evaluation Metrics for all 5 algorithms
for _ in algorithm:
    print("Algorithm used by me : " ,_)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = _)
    reg = lr.fit(X_train,y_train)
    preds = lr.predict(X_test)
    print("Mean Squared Error is :")
    print(cuml.metrics.regression.mean_squared_error(y_test.astype('int64'),preds.astype('int64')))
    print("R2 Score is :")
    print(cuml.metrics.regression.r2_score(y_test.astype('float32'),preds.astype('float32')))
    print("Mean Absolute Error is :")
    print(cuml.metrics.regression.mean_absolute_error(y_test.astype('int64'),preds.astype('int64')))
    print("\n")

In [ ]:
sample_sub = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample_sub.head()

In [ ]:
sample_sub['SalePrice']

In [ ]:
sample_sub['SalePrice'] = y_pred_split

In [ ]:
sample_sub['SalePrice']

In [ ]:
sample_sub.to_csv('submission1.csv' , index = False)